<h1> imports

In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import tqdm as tqdm
import itertools
#lda
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
from preprocessing_algorithms import *
from efficiency_scores import *

C:\Users\Specialist\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from lexrank.utils.text import tokenize
from lexrank.mappings.stopwords import STOPWORDS

<h1> Pre Processing </h1>

In [3]:
def tokenization(text):
    sentences = sent_tokenize(text)
    filtered_sentences = remove_stop_words(sentences)
    return filtered_sentences,sentences

In [4]:
def remove_stop_words(sentences):
    # define a set of stop words 
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []
    for sentence in sentences:
        # tokenize each sentence into words
        word_tokens = word_tokenize(sentence)
        # remove all stop words from each sentence
        filtered_words = [word for word in word_tokens if not word in stop_words]
        # join all filtered words back into a single sentence
        filtered_sentence = ' '.join(filtered_words)
        filtered_sentences.append(filtered_sentence)
    return filtered_sentences

<h1> Text Matching 

In [5]:
def text_matching(filtered_sentences, sentences):
    # timeNow("Starting Text Matching.")
    # summary_size = 5
    # f = open(file_name, "r")
    # text = ""
    # for line in f:
    #     text += line.replace('!','.').replace('?','.').replace('\n',' ')
    # f.close()
    
    # sentences = sent_tokenize(text)  # Tokenize the text into sentences
    # sentences = remove_stop_words(sentences)
    
    word_frequencies = {}  # Create an empty dictionary to store the word frequencies
    for sentence in filtered_sentences:  # Loop through each sentence in the text
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words
        for word in words:  # Loop through each word in the sentence
            if word not in word_frequencies.keys():  # Check if the word is already in the dictionary
                word_frequencies[word] = 1  # If not, set its count to 1
            else:
                word_frequencies[word] += 1  # If yes, increment its count by 1
    summary = []  # Create an empty list to store the summary sentences
    sentence_map = {} # Create an empty dictionary to store the sentence scores
    sent_scores = []
    sent_index = 0
    for sentence in sentences:  # Loop through each sentence in the text again
        words = nltk.word_tokenize(sentence)  # Tokenize each sentence into words again
        score = 0  # Initialize a score variable to 0
        for word in words:  # Loop through each word in the sentence
            if word in word_frequencies.keys():  # Check if the current word is present in our dictionary of word frequencies
                score += word_frequencies[word]  # If yes, add its frequency to our score variable
        sentence_map[sent_index] = [sentence]
        sent_scores.append(score)
        sent_index += 1
    
    # scores = sorted(sentence_map.keys())
    # scores = scores[len(scores)-summary_size:]
    maxScore = max(sent_scores)    
    sortedScored = list(reversed(sorted(sent_scores)))
    
    for i in range(len(sent_scores)):
        sentence_map[i].append(sent_scores[i] / maxScore)
        sentence_map[i].append(sortedScored.index(sent_scores[i]))
    
    # for score in scores:
    #     summary.append(sentence_map[score])
        # print("Score :[",score,"] Sentence :", sentence_map[score])
    # timeNow("Finished Text Matching")
    return sentence_map

<h1> Luhn

In [6]:
def luhn_algorithm(filtered_sentences, sentences):
    # timeNow("Starting Luhn")
    # Initialize a list to store the summary
    summary = []
    # # Split the text into sentences
    # sentences = text.split('.')
    # Initialize a list to store the sentence scores
    sentence_scores = []
    sentence_map = {}
    sent_index = 0
    # Iterate through each sentence
    for sentence in filtered_sentences:
        # Split the sentence into words
        words = sentence.split()
        # Initialize a score for the sentence
        score = 0
        # Iterate through each word
        for word in words:
            # Calculate the score for the word
            score += len(word)
        # Add the score to the sentence scores list
        sentence_scores.append(score)
        sentence_map[sent_index] = [sentences[sent_index]]
        sent_index += 1

    maxScore = max(sentence_scores)
    sortedScored = list(reversed(sorted(sentence_scores)))
    
    for i in range(len(sentence_scores)):
        sentence_map[i].append(sentence_scores[i] / maxScore)
        sentence_map[i].append(sortedScored.index(sentence_scores[i]))
    # timeNow("Finished Luhn")
    return sentence_map

<h1> Latent Semantic Analysis

In [7]:
def create_tf_idf(filtered_sentences):
    tfidfconverter = TfidfVectorizer()
    X = tfidfconverter.fit_transform(filtered_sentences).toarray()
    return X

In [8]:
def lsa_algorithm(X):
    svdmodel = TruncatedSVD(n_components=2)
    svdmodel.fit(X)
    result = svdmodel.transform(X)
    return result

In [9]:
def lsa_summarization(filtered_sentences, sentences):
    # timeNow("Starting LSA")
    # sentences = tokenization(text)
    # filtered_sentences = remove_stop_words(sentences)
    X = create_tf_idf(filtered_sentences)
    result = lsa_algorithm(X)
    scores = result[:,1]
    summary = ""
    sentence_map = {}
    normalized = (scores-min(scores))/(max(scores)-min(scores))
    # summarize our text by selecting only those sentences with higher scores
    sortedScored = list(reversed(sorted(normalized)))
    for i in range (len (normalized)):
        sentence_map[i] = [sentences[i], normalized[i], sortedScored.index(normalized[i])]
    # timeNow("Finished LSA")
    return sentence_map

<h1> Text Rank

In [10]:
word_embeddings = {}

In [11]:
if not(word_embeddings):
    print("Loading Big File...")
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    print("Finished Loading Big File.")
def textRank(filtered_sentences, sentences):
    # timeNow("Starting Text Rank")
    sentence_vectors = []
    for i in filtered_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100))
        sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])

    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]   
    #begining Graph stap
    import networkx as nx
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    maxScore = max(scores.values())
    for key in scores:
        scores[key] = scores[key] / maxScore
    sent_map = {}
    sortedScores = list(reversed(sorted(scores.values())))
    for index in range(len(sentences)):
        sent_map[index] = [sentences[index] , scores[index], sortedScores.index(scores[index])]
    #ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # timeNow("Finished Text Rank")
    return sent_map

Loading Big File...
Finished Loading Big File.


<h1> Lex Rank </h1>

In [12]:
def LexRank_algorithm(filtered_sentences,sentences,size=5,threshold = 0.095):
    tfidfconverter = TfidfVectorizer()
    tf_idf = tfidfconverter.fit_transform(filtered_sentences).toarray()
    length = len(tf_idf)
    similarity_matrix = np.zeros([length] * 2)
    
    for i in range(length):
        for j in range(i, length):
            similarity = cosine_similarity(tf_idf[i],tf_idf[j],i,j)

            if similarity:
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity    
    
    def get_summary(sentences,similarity_matrix,threshold,summary_size=1):

        if not isinstance(summary_size, int) or summary_size < 1:
            raise ValueError('\'summary_size\' should be a positive integer')

        lex_scores = rank_sentences(sentences,similarity_matrix,threshold)

        sorted_ix = np.argsort(lex_scores)[::-1]

        summary_index=[]
        for i in sorted_ix[:summary_size]:
            summary_index.append(i)
        #print(summary_index)
        return lex_scores,summary_index

    scores , summary_index = get_summary(sentences,similarity_matrix,threshold,size)

    sentence_map = {}
    normalized = (scores-min(scores))/(max(scores)-min(scores))
    sortedScored = list(reversed(sorted(normalized)))
    for i in range (len (normalized)):
        sentence_map[i] = [sentences[i], normalized[i], sortedScored.index(normalized[i])]
    return sentence_map

def connected_nodes(matrix):
    _, labels = connected_components(matrix)
    z = csr_matrix(matrix)
    groups = []
    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)
    return groups

def cosine_similarity(list_1, list_2,i,j):
        if i == j :
            return 1
        dot = np.dot(list_1, list_2)
        if math.isclose(dot, 0):
            return 0
        norm = (np.linalg.norm(list_1) * np.linalg.norm(list_2))
        cos_sim = dot / norm
        return cos_sim
    
    
def stationary_distribution(transition_matrix,normalized=True):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)
    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix)
        distribution[group] = eigenvector
    if normalized:
        distribution /= n_1
    return distribution

def _power_method(transition_matrix):
    sentences_count = len(transition_matrix)
    eigenvector = np.ones(sentences_count)
    if len(eigenvector) == 1:
        return eigenvector
    transposed_matrix = transition_matrix.T
    lambda_val = 1.0

    while np.allclose(lambda_val, eigenvector):
        eigenvector_next = np.dot(transposed_matrix, eigenvector)
        lambda_val = np.linalg.norm(np.subtract(eigenvector_next, eigenvector))
        eigenvector = eigenvector_next
    return eigenvector


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    return weights_matrix / row_sum

def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = weights_matrix#np.zeros(weights_matrix.shape)
    #print(discrete_weights_matrix)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1
    #print(discrete_weights_matrix)

    return create_markov_matrix(discrete_weights_matrix)

def degree_centrality_scores(similarity_matrix,threshold=None,increase_power=True):
    if not (threshold is None or isinstance(threshold, float) and 0 <= threshold < 1):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None')

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(similarity_matrix,threshold)

    scores = stationary_distribution(markov_matrix,normalized=True)
    return scores

def rank_sentences(sentences,similarity_matrix,threshold=0.03):  
    scores = degree_centrality_scores(similarity_matrix,threshold)
    return scores

<h1> LDA </h1>

In [13]:
def lda_algorithm(filtered_sentences,sentences,num_topics = 3):
    # Create a CountVectorizer object to preprocess the text
    vectorizer = CountVectorizer(stop_words='english')

    # Fit and transform the vectorizer to the text
    doc_term_matrix = vectorizer.fit_transform(filtered_sentences)

    # Create an LDA object using Gibbs sampling
    lda = LatentDirichletAllocation(n_components=num_topics, learning_method='batch', random_state=42)

    # Fit the LDA model to the document-term matrix
    lda.fit(doc_term_matrix)

    # Extract the topic summaries
    topic_word_distributions = lda.components_
    for i, topic_dist in enumerate(topic_word_distributions):
        topic_words = [vectorizer.get_feature_names_out()[j] for j in topic_dist.argsort()[:-6:-1]]
        #print(f"Topic {i}: {' '.join(topic_words)}")

    # Extract the topic distribution for each sentence
    sentence_topic_distributions = lda.transform(doc_term_matrix)

    # Calculate the sentence scores
    sentence_scores = sentence_topic_distributions.max(axis=1)

    # Sort the sentences by score in descending order
    sorted_indices = sentence_scores.argsort()[::-1]

    # Extract the top N sentences
    # top_indices = sorted_indices[:size]
    # top_sentences = [sentences[i] for i in top_indices]

    # Print the summary
    #print('\n'.join(top_sentences))
    sent_map = {}
    for i in range(len(sentences)):
        sent_map[i] = [sentences[i],sentence_scores[i],sorted_indices[i]]
        
    return sent_map

<h1> Summarizing the dataset

In [14]:
def createEnsembledic(tm_dic,luhn_dic,lsa_dic,tr_dic,lex_dic, lda_dic):
    ensemble_dic = {}
    finalScores = []
    for key in tm_dic:
        final_score = tm_dic[key][1] + luhn_dic[key][1]+ lsa_dic[key][1] + tr_dic[key][1] + lex_dic[key][1] + lda_dic[key][1]
        finalScores.append(final_score)
        scores_arr = [tm_dic[key][1],luhn_dic[key][1],lsa_dic[key][1],tr_dic[key][1],lex_dic[key][1],lda_dic[key][1]]
        sent = tm_dic[key][0]
        ensemble_dic[key] = [final_score, scores_arr, sent]
    return ensemble_dic,finalScores

In [15]:
def ensembleSummary(ensemble_dic,finalScores,size):
    finalScores = list(reversed(sorted(finalScores)))[:]
    summaryScores = finalScores[:size] # number of summary sentences
    summary = []
    for key in ensemble_dic:
        if(ensemble_dic[key][0] in summaryScores):
            summary.append(ensemble_dic[key][2])
    return summary

In [16]:
def extractSummaryFromDic(dic, size):
    summaryScores = []
    summarySent = []
    for i in range(size):
        summaryScores.append(dic[i][1])
        summarySent.append(dic[i][0])
        
    for i in range(size, len(dic), 1):
        if(dic[i][1] > min(summaryScores)):
            ind = summaryScores.index(min(summaryScores))
            summaryScores[ind] = dic[i][1]
            summarySent[ind] = dic[i][0]
    return summarySent

In [17]:
def getTextRanks(filtered_sentences, sentences):
    tm_dic = text_matching(filtered_sentences, sentences)
    luhn_dic = luhn_algorithm(filtered_sentences, sentences)
    lsa_dic = lsa_summarization(filtered_sentences, sentences)
    tr_dic = textRank(filtered_sentences, sentences)
    lex_dic = LexRank_algorithm(filtered_sentences, sentences)
    lda_dic = lda_algorithm(filtered_sentences, sentences)
    ensemble_dic, ensemble_scores = createEnsembledic(tm_dic, luhn_dic, lsa_dic, tr_dic, lex_dic, lda_dic)
    return ensemble_dic#, ensemble_scores, tm_dic, luhn_dic, lsa_dic, tr_dic

In [18]:
def get_combinations(arr):
    combinations = []
    for i in range(1,len(arr)+1):
        for element in itertools.combinations(arr, i):
            combinations.append(element)
    return combinations

In [19]:
with open ("summary input.txt", "r") as myfile:
    data = myfile.read()
    filtered_sentences,sentences = tokenization(data)

In [20]:
def allCombs(ensemble_dic, useWeights, weights):
    Algorithms = ["Tm","Lex","Luhn","Lsa","Tr","LDA"]
    lst = []
    lstCols = get_combinations(Algorithms)
    wtm = 1
    wlex = 1
    wluhn = 1
    wlsa = 1
    wtr = 1
    wlda = 1
    if(useWeights):
        wtm = weights["tm"]
        wlex =  weights["lex"]
        wluhn =  weights["luhn"]
        wlsa =  weights["lsa"]
        wtr =  weights["tr"]
        wlda =  weights["lda"]
    for i in ensemble_dic:
        tm = ensemble_dic[i][1][0] * wtm
        luhn = ensemble_dic[i][1][1] * wluhn
        lsa = ensemble_dic[i][1][2] * wlsa
        tr = ensemble_dic[i][1][3] * wtr
        lex = ensemble_dic[i][1][4] * wlex
        lda = ensemble_dic[i][1][5] * wlda
        values = [tm,lex,luhn,lsa,tr,lda]

        CombNums = get_combinations(values)
        for i,comb in enumerate(CombNums):
            CombNums[i] = sum(comb)/len(comb)
        lst.append(CombNums)
    combinationScores = pd.DataFrame(lst, columns = lstCols)     
    return combinationScores

In [21]:
def buildDF(filtered_sentences, sentences, useWeights, weights):
    ensemble_dic = getTextRanks(filtered_sentences, sentences)
    df = allCombs(ensemble_dic, useWeights, weights)
    return df

In [22]:
def summarizeWith(sentences, df, algorithm, percentage):
    if(percentage >= 1):
        sent = int(percentage)
    else:
        sent = int(percentage * len(df))
    summInds = df.nlargest(n=sent, columns=[algorithm])[algorithm].keys()
    # summInds = sorted(summInds)
    summ = ""
    for i in summInds:
        summ += sentences[i]
    return summ

In [23]:
# buildDF(filtered_sentences, sentences)

,"(Tm,)","(Lex,)","(Luhn,)","(Lsa,)","(Tr,)","(LDA,)","(Tm, Lex)","(Tm, Luhn)","(Tm, Lsa)","(Tm, Tr)",...,"(Lex, Luhn, Tr, LDA)","(Lex, Lsa, Tr, LDA)","(Luhn, Lsa, Tr, LDA)","(Tm, Lex, Luhn, Lsa, Tr)","(Tm, Lex, Luhn, Lsa, LDA)","(Tm, Lex, Luhn, Tr, LDA)","(Tm, Lex, Lsa, Tr, LDA)","(Tm, Luhn, Lsa, Tr, LDA)","(Lex, Luhn, Lsa, Tr, LDA)","(Tm, Lex, Luhn, Lsa, Tr, LDA)"
0,0.272813,0.481029,0.502793,0.310268,0.876167,0.946820,0.753842,0.775606,0.583080,1.148979,...,2.806809,2.614283,2.636048,2.443069,2.513723,3.079622,2.887096,2.908860,3.117076,3.389889
1,0.170273,0.170805,0.430168,0.242011,0.855048,0.927874,0.341078,0.600440,0.412284,1.025321,...,2.383895,2.195738,2.455101,1.868304,1.941131,2.554167,2.366011,2.625374,2.625906,2.796179
2,0.808090,0.077315,0.452514,0.378279,0.933820,0.931024,0.885405,1.260604,1.186370,1.741910,...,2.394673,2.320438,2.695637,2.650018,2.647223,3.202763,3.128528,3.503728,2.772952,3.581042
3,0.516463,1.000000,0.525140,0.691638,0.975627,0.952457,1.516463,1.041603,1.208101,1.492090,...,3.453224,3.619723,3.144862,3.708868,3.685698,3.969687,4.136185,3.661325,4.144862,4.661325
4,0.375353,0.165715,0.452514,0.358791,0.931568,0.935785,0.541068,0.827867,0.734144,1.306921,...,2.485583,2.391860,2.678658,2.283941,2.288159,2.860935,2.767213,3.054011,2.844374,3.219726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.334901,0.150730,0.469274,0.221761,0.976314,0.950128,0.485631,0.804175,0.556662,1.311215,...,2.546446,2.298933,2.617477,2.152980,2.126794,2.881347,2.633834,2.952378,2.768207,3.103108
119,0.211665,0.098111,0.636872,0.287802,0.949203,0.951949,0.309777,0.848537,0.499467,1.160868,...,2.636134,2.287065,2.825825,2.183652,2.186399,2.847799,2.498730,3.037490,2.923936,3.135601
120,0.693321,0.066966,0.720670,0.349351,0.976777,0.959071,0.760287,1.413991,1.042672,1.670097,...,2.723484,2.352165,3.005869,2.807085,2.789379,3.416805,3.045486,3.699190,3.072835,3.766156
121,0.224835,0.099321,0.413408,0.438117,0.745311,0.953355,0.324156,0.638243,0.662952,0.970146,...,2.211395,2.236103,2.550190,1.920991,2.129036,2.436230,2.460939,2.775026,2.649511,2.874347
